In [1]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
mesh = PoissonFEM.RectangularMesh(np.ones(4)/4)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
mesh.setEssentialBoundary(origin)

In [4]:
funSpace = PoissonFEM.FunctionSpace(mesh)

In [5]:
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)
K.compGlobStiffStencil()    # very slow for large systems, but precomputed, --> no bottleneck
K.compSparsityPattern()

In [6]:
# Precomputations
Kmat = PETSc.Mat().createAIJ(size=(mesh.nEq, mesh.nEq), nnz=K.nnz)
lmbda = PETSc.Vec().createSeq(mesh.nCells)
Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

In [7]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [8]:
# # Precomputations
# mesh.compGlobStiffStencil()   # very slow for large systems, but precomputed, --> no bottleneck
# mesh.compSparsityPattern()

# K0 = PETSc.Mat().createAIJ(size=(mesh.nEq, mesh.nEq), nnz=mesh.K_nnz)
# lmbda = PETSc.Vec().createSeq(mesh.nCells)
# Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
# x = PETSc.Vec().createSeq(mesh.nEq)
# rhs = PETSc.Vec().createSeq(mesh.nEq)

In [9]:
# # Set up solver
# ksp = PETSc.KSP().create()
# ksp.setType('preonly')
# pc = ksp.getPC()
# pc.setType('cholesky')
# ksp.setFromOptions() #???

In [10]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [11]:
N = 1e5

In [12]:
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    lmbda.setValues(range(mesh.nCells), np.ones(mesh.nCells))
    rhs.setValues(range(mesh.nEq), np.ones(mesh.nEq))
    K.globStiffStencil.mult(lmbda, Kvec)
    Kmat.setValuesCSR(K.indptr, K.indices, Kvec.getValues(K.vec_nonzero))
    Kmat.assemblyBegin()
    Kmat.assemblyEnd()

    # solving
    ksp.setOperators(Kmat)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  3.462363243103027e-05


In [13]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, mesh.K_indptr))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

AttributeError: 'RectangularMesh' object has no attribute 'Kvec_nonzero'

In [ ]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)